# Train, validate & Generate Submitions

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from keras import losses
import keras

from src.data import get_generators, format_bytes
from src.model import get_2losses_model, add_clf_layer, add_clf_layer_sparse, get_model, get_RAEwSC_compiled, get_RAEwSC_and_WS_compiled
from src.losses import cross_entropy_with_axis, model_evaluate, write_submission_files

from keras.utils.vis_utils import plot_model
# plot_model(model, show_shapes=True, show_layer_names=True)
# start tensorboard: tensorboard --logdir .

Using TensorFlow backend.


In [109]:
# define global names
EXTRA_DATA_MODEL, EXTRA_DATA_MODELwIN, RAEwSCwWSwINwCLF = "RAE_w_SC_WS", "RAE_w_SC_WS_wIN", "RAEwSCwWSwINwCLF"

# define model and city
MODEL_TYPES = {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: EXTRA_DATA_MODEL, 4: EXTRA_DATA_MODELwIN, 
               5: RAEwSCwWSwINwCLF}
CITIES = {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
DATA_SPLIT = {0: "non-overlapping", 1: "all_possible_slots", 2: "like-test"}

############################################# DEFINE HYPERPARAMETERS
DATA_SPLIT_IDX = 2 # choose how to split data
params = {
    # data params
    "batch_size": 4,
    "length_seq_in": 3, #3, # Set up when calling a model
    "length_seq_out": 3,
    "batch_size_validation": 48,
    "data_split": DATA_SPLIT[DATA_SPLIT_IDX], # Set up when calling a model
    "batch_size_test": 5,
    # loss params
    'optimizer': 'adam',  # adam or sgd
    'learning_rate': 0.001,
    'decay_rate': 0.9999,  # Learning rate decay per minibatch.
    'min_learning_rate': .0000001,  # Minimum learning rate.
}

############################################# DEFINE MODEL WEIGHTS
models_weight = {
    "ConvLSTM": { # _weights_e04_valoss0.009_relu 
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_model_ep_2.h5".format("Moscow"), 
        "Istanbul": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_weights_e03_valoss0.009_relu3.hdf5".format("Istanbul"), 
        "Berlin": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_model_ep_5.h5".format("Berlin"),
        "Model_definition_file": "{}/projects/nips_traffic/src/models.py".format("/home/pherruzo"),
        "Model_definition_function": "build_model" 
    },
    
    "ConvLSTM+Clf": { # _weights_e04_valoss0.025_relu.hdf5, _weights_e04_valoss0.025_relu2.hdf
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_weights_e04_valoss0.025_relu2.hdf5".format("Moscow"),
        "Istanbul": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_weights_06-1.04.hdf5".format("Istanbul"), # weights_Istanbul_06-1.04.hdf5, weights_Istanbul_04-1.05.hdf5, weights_Istanbul_03-1.05.hdf5  
        "Berlin": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_model_ep_1_reg_clf_cont.h5".format("Berlin")
    },
    
    "RAE_w_SC": { # best: _weights_e03_valoss0.012_raewsc_sgd | prev: _weights_e02_valoss0.014_raewsc_sgd_emb
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_weights_e03_valoss0.012_raewsc_sgd.hdf5".format("Moscow"),
        "Istanbul": None, 
        "Berlin": None
    },
    
    EXTRA_DATA_MODEL: { # best: _e02_valoss0.012_exo_vars, best in test-like: _e91_valoss0.012_just_try 
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_e91_valoss0.012_just_try.hdf5".format("Moscow"), # works good for num_seq_in=3
        "Istanbul": None,
        "Berlin": None
    },
    
    EXTRA_DATA_MODELwIN: { # best (all slots): _e04_valoss0.012_RAEwSCwWTwIN_new_all_slots, # like-test: _e18_valoss0.012_RAEwSCwWTwINz_all_slots_like_test
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_e18_valoss0.012_RAEwSCwWTwINz_all_slots_like_test.hdf5".format("Moscow"),
        "Istanbul": None,
        "Berlin": None
    },
    
    RAEwSCwWSwINwCLF: { # best (all slots): _e19_valoss0.502_RAEwSCwWSwINwCLF #  _e12_valoss1.294_RAEwSCwWSwINwCLF
        "Moscow": "/home/pherruzo/projects/nips_traffic/checkpoints/{}_e44_valoss0.026_RAEwSCwWSwINwCLF.hdf5".format("Moscow"),
        "Istanbul": None,
        "Berlin": None,
        "loss_type": [None, 'Sparse'][1]#'Categorical'
    }
}

############################################# PREPARE POSSIBLE MODELS
def get_a_model(model_type, training_params):
    """ model_type in ["ConvLSTM+Clf", "ConvLSTM", "RAE_w_SC"] """
    
    if model_type=="ConvLSTM+Clf":
        return get_2losses_model(model_path=models_weight[training_params["previus_model"]]["Model_definition_file"], 
                                model_name=models_weight[training_params["previus_model"]]["Model_definition_function"], 
                                weights_initial_model=None, 
                                weights_current_model=models_weight[training_params["current_model"]][training_params["city"]], 
                                add_model=add_clf_layer_sparse, 
                                add_loss=losses.sparse_categorical_crossentropy,
                                sample_weight_mode="temporal")
    
    elif model_type=="ConvLSTM":
        from keras import optimizers
        return get_model(model_path=models_weight[training_params["previus_model"]]["Model_definition_file"], 
                         function_name=models_weight[training_params["previus_model"]]["Model_definition_function"], 
                         weights=models_weight[training_params["previus_model"]][training_params["city"]], 
                         opt=optimizers.Adam(lr=0.0001), loss=losses.mean_squared_error)
    
    elif model_type=="RAE_w_SC":
        return get_RAEwSC_compiled(weights=models_weight[model_type][training_params["city"]], 
                                   lr=params['learning_rate'])
    elif model_type==EXTRA_DATA_MODEL:
        dropout_enc, b_norm_enc = 0.04, True
        dropout_dec, b_norm_dec = 0.03, True
        print("dropout_enc:", dropout_enc, "dropout_dec:", dropout_dec)
        # get basic model def get_RAEwSC_and_WS_compiled(weights=None, lr = 0.001, grad_clip=1., loss_weights={'predicted_frames':1., 'predicted_emb':1.}):
        return get_RAEwSC_and_WS_compiled(weights=models_weight[model_type][training_params["city"]], lr=params['learning_rate'],
                                          loss_weights={'predicted_frames':1., 'predicted_emb':1.},
                                           dropout_enc=dropout_enc, dropout_dec=dropout_dec, 
                                           b_norm_enc=b_norm_enc, b_norm_dec=b_norm_dec)
    elif model_type==EXTRA_DATA_MODELwIN:
        from src.models.RAEwSCwWSwIN import get_RAEwSC_and_WS_compiled_w_input
        
        loss_w = {'predicted_frames':1., 'predicted_emb':0.9}
        
        dropout_enc = 0.05
        dropout_dec = 0.03
        
        print("loading {} ...".format(model_type))
        print("dropout_enc:", dropout_enc, "dropout_dec:", dropout_dec)
        print("loss weights:", loss_w)
        return get_RAEwSC_and_WS_compiled_w_input(weights=models_weight[model_type][training_params["city"]], length_seq_in=params['length_seq_in'], 
                                                  lr=params['learning_rate'], loss_weights=loss_w,
                                                  dropout_enc=dropout_enc, dropout_dec=dropout_dec, for_new_model=False)
    
    elif model_type==RAEwSCwWSwINwCLF:
        from src.models.RAEwSCwWSwINxCLF import get_RAEwSCwWSwIwCLF_compiled as get_model
        
        loss_w = {'predicted_frames':0., 'predicted_emb':1.}
        
        dropout_enc = 0.05
        dropout_dec = 0.03
        
        print("loading {} ...".format(model_type))
        print("dropout_enc:", dropout_enc, "dropout_dec:", dropout_dec)
        print("loss weights:", loss_w)
        return get_model(weights=models_weight[model_type][training_params["city"]], length_seq_in=params['length_seq_in'], 
                         lr=params['learning_rate'], loss_weights=loss_w,
                         dropout_enc=dropout_enc, dropout_dec=dropout_dec)
    # get_RAEwSCwWSwIwCLF_compiled

############################################# DEFINE TRAINING PARAMS
def get_training_params(model_type, city):
    """ model_type in ["ConvLSTM", "ConvLSTM+Clf", "RAE_w_SC", "RAE_w_SC_WS"] 
        city in ['Moscow', 'Istanbul', 'Berlin']
    """
    training_params = {
        "city": city,
        "previus_model": "ConvLSTM",     # used in old methods, NOT TO CHANGE todo: replace
        "current_model": "ConvLSTM+Clf", # used in old methods, NOT TO CHANGE todo: replace
        "tensorboard_path": "/home/pherruzo/projects/nips_traffic/tensorboard/",
        "log_path": "/home/pherruzo/projects/nips_traffic/log_files/",
        "model_type": model_type,
        "output_dir": "/home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt"
    }
    return training_params

############################################# LOAD DATA GENERATOR
def load_model_and_data(model_type, city, params, length_seq_in=None, data_split=None, debug=False):
    
    if length_seq_in is not None:
        params["length_seq_in"] = length_seq_in
        
    if data_split is not None:
        params["data_split"] = DATA_SPLIT[data_split]
        
    # load training params
    training_params = get_training_params(model_type, city)
    
    # load data
    if "loss_type" in models_weight[model_type]:
        loss_type = models_weight[model_type]["loss_type"]
        print("---> loss type for clf:", loss_type)
    else:
        loss_type = None
    training_ds, val_ds, test_ds = get_generators(training_params["model_type"],   params["batch_size"], 
                                                  training_params["city"],         params["length_seq_in"], 
                                                  params["batch_size_validation"], data_split=params["data_split"], debug=debug, loss_type=loss_type)
    # load model
    model = get_a_model(model_type, training_params)
    
    # print info
    print("")
    print("· length_seq_in:", params["length_seq_in"])
    print("· data_split:", params["data_split"])
    print("· model_type:", training_params["model_type"])
    print("· city      :", training_params["city"])
    print(" 1 epoch training is {} batches, validation is {}, and test is {}.".format(len(training_ds), len(val_ds), len(test_ds)))
    
    return training_ds, val_ds, test_ds, model, training_params

def plot(keys_list, history):

    leg = []
    for k in keys_list:
        plt.plot(history.history[k], '-o')
        leg.append(k)
    
    plt.title(leg)
    plt.xlabel('epoch')
    plt.legend(leg, loc='center')#'upper left')
    plt.show()

## TRAINING

### 1. Load model & data

In [5]:
#### load data and model --> 
# choose model          {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: "RAE_w_SC_WS", 4: "RAE_w_SC_WS_wIN", 5: "RAEwSCwWSwINwCLF"}
# choose city           {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
# choose data-split     {0: "non-overlapping", 1: "all_possible_slots", 2: "like-test"}
# choose length_seq_in  3, 4, 5, 6, 7, 8, 9, 10, 11, 12

params['learning_rate'] = 0.01
debug = False

m_type, city = 4, 0
data_split_idx = 2
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

-->> Data has been shuffled in: training


W1022 09:13:58.139183 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1022 09:13:58.147670 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1022 09:13:58.150118 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1022 09:13:58.166394 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_se

loading RAE_w_SC_WS_wIN ...
dropout_enc: 0.05 dropout_dec: 0.03
loss weights: {'predicted_frames': 1.0, 'predicted_emb': 0.9}


W1022 09:13:58.648825 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W1022 09:13:58.723539 140354142054208 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1022 09:13:58.726788 140354142054208 deprecation.py:506] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Encoder output shape: (?, 8, 8, 32) reshaped: (?, ?)
Decoder input shape: (?, 2048) reshaped: (?, 8, 8, 32)
embeddings.shape: (?, 6, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 6, 2388)
FC before recurrent embeddings.shape: (?, 6, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e18_valoss0.012_RAEwSCwWTwINz_all_slots_like_test.hdf5
Freezing layers...

· length_seq_in: 6
· data_split: like-test
· model_type: RAE_w_SC_WS_wIN
· city      : Moscow
 1 epoch training is 357 batches, validation is 5, and test is 72.


### 2. Define callbacks

In [114]:
from keras import backend as K 

class LRTensorBoard(keras.callbacks.TensorBoard):
    def __init__(self, log_dir, **kwargs):  # add other arguments to __init__ if you need
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs.update({'lr': K.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)
        
############################################# PREPARE CALLBACKS
run_name = 'tttest'#'RAEwSCaWT_w_IN'
run_num = '1'
## tensorboard
tb_folder = "/tb_{}_{}/run_{}".format(training_params["city"],"1", run_name+'_'+run_num)
tb_folder = training_params["tensorboard_path"]+MODEL_TYPES[m_type]+tb_folder
# tb_update_freq = 1000 * params["batch_size"]
# print("Saving training progress in {} each {} samples.".format(tb_folder, tb_update_freq))

# save best model
# checkpoint
root= "/home/pherruzo" 
model_dir=root+"/projects/nips_traffic/checkpoints/"
filepath=model_dir+training_params["city"]+"_e{epoch:02d}_valoss{val_loss:.3f}_"+run_name+".hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_weights_only=True, save_best_only=True)

# ep 1-4 lr: 0.001
# ep 5- lr: 0.0001

my_callbacks = [keras.callbacks.TensorBoard(log_dir=tb_folder), checkpoint, LRTensorBoard(log_dir=tb_folder)] #, update_freq=tb_update_freq)]
# my_callbacks = [keras.callbacks.TensorBoard(log_dir=tb_folder, histogram_freq=1, write_grads=True), checkpoint, LRTensorBoard(log_dir=tb_folder]
# my_callbacks = None
filepath, tb_folder

('/home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e{epoch:02d}_valoss{val_loss:.3f}_tttest.hdf5',
 '/home/pherruzo/projects/nips_traffic/tensorboard/RAE_w_SC_WS_wIN/tb_Moscow_1/run_tttest_1')

# TRAIN

In [ ]:
#K.set_value(model.optimizer.lr, 0.01)

In [ ]:
DEBUG = False

if DEBUG:
    print('validation data to memory')
    valid_data = ([x_val, y_val], y_val)
    my_callbacks = [keras.callbacks.TensorBoard(log_dir=tb_folder, histogram_freq=1, write_grads=True), 
                    checkpoint, 
                    LRTensorBoard(log_dir=tb_folder)]
else:
    valid_data = val_ds
    my_callbacks = [keras.callbacks.TensorBoard(log_dir=tb_folder), checkpoint, LRTensorBoard(log_dir=tb_folder)]

print("learning rate:", K.get_value(model.optimizer.lr))
hist = model.fit_generator(training_ds, epochs=20, verbose=1, callbacks=my_callbacks, 
                           validation_data=valid_data,
                           max_queue_size=10, workers=4, shuffle=True, initial_epoch=1)#, steps_per_epoch=2000) # starts with next epoch

learning rate: 0.01
Epoch 2/20
2280/2280 [==============================] - 1806s 792ms/step - loss: 0.0121 - mean_squared_error: 0.0121 - r_emb_loss: 2.0438e-08 - val_loss: 0.0120 - val_mean_squared_error: 0.0120 - val_r_emb_loss: 3.2361e-09
Epoch 3/20
2280/2280 [==============================] - 1801s 790ms/step - loss: 0.0121 - mean_squared_error: 0.0121 - r_emb_loss: 2.0797e-08 - val_loss: 0.0120 - val_mean_squared_error: 0.0120 - val_r_emb_loss: 6.6008e-09
Epoch 4/20
2280/2280 [==============================] - 1808s 793ms/step - loss: 0.0121 - mean_squared_error: 0.0121 - r_emb_loss: 2.1595e-08 - val_loss: 0.0119 - val_mean_squared_error: 0.0119 - val_r_emb_loss: 3.9253e-08
Epoch 5/20
2280/2280 [==============================] - 1804s 791ms/step - loss: 0.0121 - mean_squared_error: 0.0121 - r_emb_loss: 2.0826e-08 - val_loss: 0.0119 - val_mean_squared_error: 0.0119 - val_r_emb_loss: 3.6410e-10
Epoch 6/20
1169/2280 [==============>...............] - ETA: 14:36 - loss: 0.0120 - mean

In [24]:
hist.history.keys()

dict_keys(['val_loss', 'val_Concat_predicted_frames_loss', 'val_softmax_clf_loss', 'val_Concat_predicted_frames_mean_squared_error', 'val_softmax_clf_sparse_categorical_crossentropy', 'loss', 'Concat_predicted_frames_loss', 'softmax_clf_loss', 'Concat_predicted_frames_mean_squared_error', 'softmax_clf_sparse_categorical_crossentropy', 'lr'])

## VALIDATION

## Evaluation of the first model ConvLSTM vs. ConvLSTM+Clf

### Moscow

In [18]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!! enviando cada Heading a un valor    !!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_model_ep_2.h5
· data_split: non-overlapping
· model_type: ConvLSTM
· city      : Moscow
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Moscow shape: (14, 3, 3)
mean mse: 0.012643993
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.5058866e-05' '0.00558

#### M1: ConvLSTM

In [13]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_model_ep_2.h5
· data_split: non-overlapping
· model_type: ConvLSTM
· city      : Moscow
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Moscow shape: (14, 3, 3)
mean mse: 0.012643993
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.5058866e-05' '0.00558

#### M2: ConvLSTM+Clf

In [5]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 1, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs new: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e04_valoss0.025_diff_slots.hdf5
temporal
· data_split: all_possible_slots
· model_type: ConvLSTM+Clf
· city      : Moscow
 1 epoch training is 3427 batches, validation is 7, and test is 8.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.012121401
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.7021246e-05' '0.005336735' '0.030951554']
 ['10 minutes' '9.086769e-05' '0.005297211' '0.030895546']
 ['15 minutes' '9.0647016e-05' '0.005320187' '0.031022849']]
-----------------------------
mean mse like submission: 0.012121498
[['        ' '    Speed   ' '   Volume  ' '   H

In [5]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 1, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

W0930 09:33:06.677484 140628157896512 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 09:33:06.685691 140628157896512 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 09:33:06.686591 140628157896512 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 09:33:06.747815 140628157896512 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W0930 09:33:08.156661 140628157896512 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs new: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e04_valoss0.025_relu.hdf5
temporal
lr: 0.0001
· data_split: all_possible_slots
· model_type: ConvLSTM+Clf
· city      : Moscow
 1 epoch training is 3427 batches, validation is 7, and test is 8.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.012060078
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.504161e-05' '0.005294468' '0.030697852']
 ['10 minutes' '8.986601e-05' '0.00525921' '0.030810388']
 ['15 minutes' '8.995142e-05' '0.0052834027' '0.03093051']]
-----------------------------
mean mse like submission: 0.012059943
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.553785e-05

In [3]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 1, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

W1002 08:50:39.688166 140712892053312 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1002 08:50:39.696547 140712892053312 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1002 08:50:39.697452 140712892053312 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1002 08:50:39.764677 140712892053312 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W1002 08:50:41.210492 140712892053312 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs new: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e04_valoss0.025_relu2.hdf5
temporal
lr: 0.0001
· data_split: all_possible_slots
· model_type: ConvLSTM+Clf
· city      : Moscow
 1 epoch training is 3427 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.012037375
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.491112e-05' '0.005278308' '0.030639661']
 ['10 minutes' '8.9811576e-05' '0.0052422746' '0.030765565']
 ['15 minutes' '8.976435e-05' '0.005265665' '0.030880405']]
-----------------------------
mean mse like submission: 0.012037826
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.53909

#### M3: RAE_w_SC

In [10]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 2, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

# here 3 ep after 7 previous ep

-->> Data has been shuffled in: training
embeddings.shape: (?, 3, 2048)
future_embeddings.shape: (?, 3, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e03_valoss0.012_raewsc_sgd.hdf5
lr: 0.0001
· data_split: all_possible_slots
· model_type: RAE_w_SC
· city      : Moscow
 1 epoch training is 20164 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.011781985
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.933762e-05' '0.0050940025' '0.029733649']
 ['10 minutes' '0.00010692001' '0.00514298' '0.030180553']
 ['15 minutes' '0.000106906

In [8]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 2, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
embeddings.shape: (?, 3, 2048)
future_embeddings.shape: (?, 3, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e07_valoss0.012_raewsc.hdf5
lr: 0.0001
· data_split: all_possible_slots
· model_type: RAE_w_SC
· city      : Moscow
 1 epoch training is 20164 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.011783949
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.911314e-05' '0.005091771' '0.02972464']
 ['10 minutes' '0.00010690344' '0.005143501' '0.030185902']
 ['15 minutes' '0.00010749385' '

## Best model performance on "like-test" split

In [16]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 2, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
embeddings.shape: (?, 3, 2048)
future_embeddings.shape: (?, 3, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_weights_e03_valoss0.012_raewsc_sgd.hdf5
lr: 0.01
· data_split: like-test
· model_type: RAE_w_SC
· city      : Moscow
 1 epoch training is 357 batches, validation is 1, and test is 72.
--> Evaluating model with keras:
[0.012460684403777122, 0.012391732074320316, 6.89523876644671e-05]
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/1
city: Moscow shape: (1, 3, 3)
mean mse: 0.012322592
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00010845185' '0.0051686806' '0.031293485']
 ['10 minutes' '0.00011554455' '0.0051963837' '0.03172064']
 ['15 minutes' '0.00011273609' '0.0052599995' '0.03192741']]
-----------------------------
mean mse like submission: 0.012392129
[['      

In [30]:
#######
####### Here is tested with the real test bins. Above I was wrong and they were other time bins (close to the real but still diff)
#######


# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 2, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])



--> Evaluating model with keras:
[0.012465481646358967, 0.01239652931690216, 6.89523876644671e-05]
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/1
city: Moscow shape: (1, 3, 3)
mean mse: 0.012327139
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.000108456465' '0.0051201344' '0.03127118']
 ['10 minutes' '0.00011458512' '0.0051807207' '0.03177587']
 ['15 minutes' '0.00011655796' '0.0052235113' '0.032033242']]
-----------------------------
mean mse like submission: 0.01239688
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.000109011504' '0.0051373923' '0.03146054']
 ['10 minutes' '0.000115180665' '0.0051983427' '0.031968005']
 ['15 minutes' '0.00011718102' '0.0052412394' '0.032225024']]
Acc y_hat in HEADING: 0.4588287417645216


#### M4: RAE_w_SC_WS

In [7]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: "RAE_w_SC_WS"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 3, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.011778687
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.8760844e-05' '0.0050893747' '0.029712154']
 ['10 minutes' '0.000106546446' '0.0051415437' '0.030173296']
 ['15 minutes' '0.00010698004' '0.0051786816' '0.030400863']]
-----------------------------
mean mse like submission: 0.011875393
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.93905e-05' '0.005116369' '0.029971529']
 ['10 minutes' '0.00010721101' '0.005168676' '0.030436058']
 ['15 minutes' '0.00010765677' '0.0052058576' '0.030665789']]
Acc y_hat in HEADING: 0.4530881848924662


In [25]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: "RAE_w_SC_WS"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 3, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
dropout_enc: 0.05 dropout_dec: 0.05
embeddings.shape: (?, 3, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 3, 2388)
FC before recurrent embeddings.shape: (?, 3, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e02_valoss0.012_exo_vars.hdf5
lr: 0.01 optimizer: <keras.optimizers.Adam object at 0x7fa5c83d3940>
· data_split: like-test
· model_type: RAE_w_SC_WS
· city      : Moscow
 1 epoch training is 357 batches, validation is 1, and test is 72.
Evaluating time-bin/time-slot batch 1/1
city: Moscow shape: (1, 3, 3)
mean mse: 0.012315642
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.0001076026' '0.0051608565' '0.031261858']
 ['10 minutes' '0.00011494466' '0.005190074' '0.031705447']
 ['15 minutes' '0.000112849455' '0.0052564978' '0.031930648']]
-----------------------------
mean

## Sistematic comparation of rae_w_sc_ws

In [14]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 3, 0
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

-->> Data has been shuffled in: training
dropout_enc: 0.05 dropout_dec: 0.05
embeddings.shape: (?, 3, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 3, 2388)
FC before recurrent embeddings.shape: (?, 3, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e02_valoss0.012_exo_vars.hdf5
lr: 0.01 optimizer: <keras.optimizers.Adam object at 0x7fa61bdf79b0>
· data_split: like-test
· model_type: RAE_w_SC_WS
· city      : Moscow
 1 epoch training is 357 batches, validation is 1, and test is 72.


## Validating over Validation Data Split: "non-overlapping"

In [13]:
root = '/home/pherruzo/projects/nips_traffic/checkpoints/'
weights = ['Moscow_e02_valoss0.012_exo_vars.hdf5', 'Moscow_e01_valoss0.013_wt.hdf5', 'Moscow_e02_valoss0.012_wt.hdf5', 
           'Moscow_e03_valoss0.012_wt.hdf5', 'Moscow_e04_valoss0.012_wt.hdf5', 'Moscow_e05_valoss0.012_wt.hdf5']
best_mse = 1.
best_model = ''
for w in weights:
    print("\n=========================================")
    print('--> loading weitghs:', w)
    model.load_weights(root+w, by_name=True)

    print("--> Evaluating model with keras:")
    mse_image_and_emb = model.evaluate_generator(val_ds)
    print(mse_image_and_emb)
    
    if mse_image_and_emb[1] < best_mse:
        best_mse = mse_image_and_emb[1]
        best_model = w

    print("--> Evaluating model with my framework:")
    model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])
print("")
print("---> Min mse: {} | Model: {}".format(best_mse, best_model))


--> loading weitghs: Moscow_e02_valoss0.012_exo_vars.hdf5
--> Evaluating model with keras:
[0.011876085240926062, 0.011875380761921406, 7.044652323072244e-07]
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Moscow shape: (7, 3, 3)
mean mse: 0.011778668
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.876041e-05' '0.0050893542' '0.029712116']
 ['10 minutes' '0.000106546584' '0.005141485' '0.03017341']
 ['15 minutes' '0.00010698072' '0.0051786355' '0.030400718']]
-----------------------------
mean mse like submission: 0.011875369
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.938991e-05' '0.0051163635' '0.029971553']
 ['10 minutes' '0.00010721106' '0.00516862

## Validating over Validation Data Split: "test-like"

In [ ]:
root = '/home/pherruzo/projects/nips_traffic/checkpoints/'
weights = ['Moscow_e02_valoss0.012_exo_vars.hdf5', 'Moscow_e01_valoss0.013_wt.hdf5', 'Moscow_e02_valoss0.012_wt.hdf5', 
           'Moscow_e03_valoss0.012_wt.hdf5', 'Moscow_e04_valoss0.012_wt.hdf5', 'Moscow_e05_valoss0.012_wt.hdf5']
best_mse = 1.
best_model = ''
for w in weights:
    print("\n=========================================")
    print('--> loading weitghs:', w)
    model.load_weights(root+w, by_name=True)

    print("--> Evaluating model with keras:")
    mse_image_and_emb = model.evaluate_generator(val_ds)
    print(mse_image_and_emb)
    
    if mse_image_and_emb[1] < best_mse:
        best_mse = mse_image_and_emb[1]
        best_model = w

    print("--> Evaluating model with my framework:")
    model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

print("")
print("---> Min mse: {} | Model: {}".format(best_mse, best_model))

#### M5: RAE_w_SC_WS_wIN

In [ ]:
params['learning_rate'] = 0.01
debug = False

m_type, city = 4, 0
data_split_idx = 2
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

In [34]:
print("\n=========================================")

print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])


--> Evaluating model with keras:
[0.012540066614747047, 0.012540065683424473, 5.502912570065632e-10]
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/1
city: Moscow shape: (1, 3, 3)
mean mse: 0.012464177
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00011239189' '0.0053015905' '0.031504516']
 ['10 minutes' '0.000117650496' '0.0053512575' '0.03199256']
 ['15 minutes' '0.000119427255' '0.0054019815' '0.03227622']]
-----------------------------
mean mse like submission: 0.012540261
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.000113290065' '0.005317206' '0.03171509']
 ['10 minutes' '0.000118585995' '0.0053668665' '0.03220445']
 ['15 minutes' '0.00012037671' '0.005417869' '0.032488614']]
Acc y_hat in HEADING: 0.4602828635856159


In [ ]:
mean mse: 0.012464179
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00011239186' '0.0053015915' '0.031504527']
 ['10 minutes' '0.00011765045' '0.0053512566' '0.03199256']
 ['15 minutes' '0.00011942723' '0.00540198' '0.03227623']]
-----------------------------
mean mse like submission: 0.01254026
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00011329015' '0.0053172046' '0.031715088']
 ['10 minutes' '0.000118586075' '0.0053668655' '0.032204457']
 ['15 minutes' '0.00012037677' '0.005417869' '0.032488607']]


##### Trained with all data
###### Eval in all data:

In [113]:
#### load data and model --> 
# choose model          {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: "RAE_w_SC_WS", 4: "RAE_w_SC_WS_wIN"}
# choose city           {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
# choose data-split     {0: "non-overlapping", 1: "all_possible_slots", 2: "like-test"}
# choose length_seq_in  3, 4, 5, 6, 7, 8, 9, 10, 11, 12

params['learning_rate'] = 0.01
debug = False

m_type, city = 4, 0
data_split_idx = 0
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

-->> Data has been shuffled in: training
loading RAE_w_SC_WS_wIN ...
dropout_enc: 0.05 dropout_dec: 0.03
loss weights: {'predicted_frames': 1.0, 'predicted_emb': 0.9}
Encoder output shape: (?, 8, 8, 32) reshaped: (?, ?)
Decoder input shape: (?, 2048) reshaped: (?, 8, 8, 32)
embeddings.shape: (?, 6, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 6, 2388)
FC before recurrent embeddings.shape: (?, 6, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e18_valoss0.012_RAEwSCwWTwINz_all_slots_like_test.hdf5
lr: 0.01 optimizer: <keras.optimizers.Adam object at 0x7f9d1c4e2e48>

· length_seq_in: 6
· data_split: non-overlapping
· model_type: RAE_w_SC_WS_wIN
· city      : Moscow
 1 epoch training is 2280 batches, validation is 5, and test is 72.


In [111]:
print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

--> Evaluating model with keras:
[0.011863351466932468, 0.011863351466932468, 1.7734439076152076e-13]
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/5
Evaluating time-bin/time-slot batch 2/5
Evaluating time-bin/time-slot batch 3/5
Evaluating time-bin/time-slot batch 4/5
Evaluating time-bin/time-slot batch 5/5
city: Moscow shape: (5, 3, 3)
mean mse: 0.011725507
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.476603e-05' '0.005103347' '0.02954914']
 ['10 minutes' '0.000101131714' '0.0051477193' '0.02997674']
 ['15 minutes' '0.00010321515' '0.00518856' '0.03026494']]
-----------------------------
mean mse like submission: 0.011816756
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '9.5909214e-05' '0.005128963' '0.029793534']
 ['10 minutes' '0.000102340266' '0.0051734922' '0.030223647']
 ['15 minutes' '0.00010444787' '0.0052143666' '0.03051411']]
Acc y_hat in HEADING: 0.4375281034092196


###### Eval in like-test:

In [10]:
params['learning_rate'] = 0.01
debug = False

m_type, city = 4, 0
data_split_idx = 2
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading RAE_w_SC_WS_wIN ...
dropout_enc: 0.05 dropout_dec: 0.03
loss weights: {'predicted_frames': 1.0, 'predicted_emb': 0.9}
Encoder output shape: (?, 8, 8, 32) reshaped: (?, ?)
Decoder input shape: (?, 2048) reshaped: (?, 8, 8, 32)
embeddings.shape: (?, 6, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 6, 2388)
FC before recurrent embeddings.shape: (?, 6, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e07_valoss0.012_RAEwSCwWTwIN_new_all_slots.hdf5
lr: 0.01 optimizer: <keras.optimizers.Adam object at 0x7f48e7092da0>

· length_seq_in: 6
· data_split: like-test
· model_type: RAE_w_SC_WS_wIN
· city      : Moscow
 1 epoch training is 357 batches, validation is 1, and test is 72.
--> Evaluating model with keras:
[0.012378418818116188, 0.012378418818116188, 2.9945559920641074e-11]
--> Evaluating model with

In [ ]:
params['learning_rate'] = 0.01
debug = False

m_type, city = 4, 0
data_split_idx = 2
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

print("--> Evaluating model with keras:")
mse_image_and_emb = model.evaluate_generator(val_ds)
print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

## M6: 

In [104]:
#### load data and model --> 
# choose model          {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC", 3: "RAE_w_SC_WS", 4: "RAE_w_SC_WS_wIN", 5: "RAEwSCwWSwINwCLF"}
# choose city           {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
# choose data-split     {0: "non-overlapping", 1: "all_possible_slots", 2: "like-test"}
# choose length_seq_in  3, 4, 5, 6, 7, 8, 9, 10, 11, 12

params['learning_rate'] = 0.01
debug = False

m_type, city = 5, 0
data_split_idx = 0
length_seq_in = 6

training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params, 
                                                                           length_seq_in=length_seq_in, data_split=data_split_idx, debug=debug)

---> loss type for clf: Sparse
-->> Data has been shuffled in: training
loading RAEwSCwWSwINwCLF ...
dropout_enc: 0.05 dropout_dec: 0.03
loss weights: {'predicted_frames': 0.0, 'predicted_emb': 1.0}
Encoder output shape: (?, 8, 8, 32) reshaped: (?, ?)
Decoder input shape: (?, 2048) reshaped: (?, 8, 8, 32)
embeddings.shape: (?, 6, 2048)
future_embeddings.shape: (?, 3, 2048)
concatenation of all inputs: (?, 6, 2388)
FC before recurrent embeddings.shape: (?, 6, 2048)
recurrent embeddings.shape: (?, ?, 2048)
prediced_frames.shape: (?, 3, 495, 436, 3)
clfs.shape: (?, ?, 5)
Freezing layer clf_layer_conv1
Freezing layer clf_layer_BN1
Freezing layer clf_layer_conv2
Freezing layer clf_layer_BN2
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Moscow_e44_valoss0.026_RAEwSCwWSwINwCLF.hdf5
Freezing layer clf_layer_conv1
Freezing layer clf_layer_BN1
Freezing layer clf_layer_conv2
Freezing layer clf_layer_BN2
lr: 0.01 optimizer: <keras.optimizers.Adam object at 0x7f9d4b573ac8>

· le

In [105]:
print("--> Evaluating model with keras:")
# mse_image_and_emb = model.evaluate_generator(val_ds)
# print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

--> Evaluating model with keras:
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/28
Evaluating time-bin/time-slot batch 2/28
Evaluating time-bin/time-slot batch 3/28
Evaluating time-bin/time-slot batch 4/28
Evaluating time-bin/time-slot batch 5/28
Evaluating time-bin/time-slot batch 6/28
Evaluating time-bin/time-slot batch 7/28
Evaluating time-bin/time-slot batch 8/28
Evaluating time-bin/time-slot batch 9/28
Evaluating time-bin/time-slot batch 10/28
Evaluating time-bin/time-slot batch 11/28
Evaluating time-bin/time-slot batch 12/28
Evaluating time-bin/time-slot batch 13/28
Evaluating time-bin/time-slot batch 14/28
Evaluating time-bin/time-slot batch 15/28
Evaluating time-bin/time-slot batch 16/28
Evaluating time-bin/time-slot batch 17/28
Evaluating time-bin/time-slot batch 18/28
Evaluating time-bin/time-slot batch 19/28
Evaluating time-bin/time-slot batch 20/28
Evaluating time-bin/time-slot batch 21/28
Evaluating time-bin/time-slot batch 22/28
Evaluating t

In [103]:
# like-test

print("--> Evaluating model with keras:")
# mse_image_and_emb = model.evaluate_generator(val_ds)
# print(mse_image_and_emb)

print("--> Evaluating model with my framework:")
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

--> Evaluating model with keras:
--> Evaluating model with my framework:
Evaluating time-bin/time-slot batch 1/5
Evaluating time-bin/time-slot batch 2/5
Evaluating time-bin/time-slot batch 3/5
Evaluating time-bin/time-slot batch 4/5
Evaluating time-bin/time-slot batch 5/5
city: Moscow shape: (5, 3, 3)
mean mse: 0.0149290245
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00021393457' '0.0067121834' '0.037486415']
 ['10 minutes' '0.00021237969' '0.0067554126' '0.037928812']
 ['15 minutes' '0.00020970749' '0.0068048937' '0.03803748']]
-----------------------------
mean mse like submission: 0.014928271
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00021133879' '0.006708829' '0.037493866']
 ['10 minutes' '0.00020962332' '0.0067517357' '0.037931755']
 ['15 minutes' '0.00020689824' '0.0068009854' '0.03803941']]
-----------------------------
mean mse with HEADING as clf: 0.022994768
[['        ' '    Speed   ' '   Volume  ' '   Heading

#### After retraining in like-data 
##### test in like-data

### Istanbul
#### I1: ConvLSTM

In [14]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Istanbul_model_ep_2.h5
· data_split: non-overlapping
· model_type: ConvLSTM
· city      : Istanbul
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Istanbul shape: (14, 3, 3)
mean mse: 0.009468561
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.484252e-05' '0.

In [3]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

W1003 09:21:30.699272 139962008799040 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 09:21:30.700322 139962008799040 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1003 09:21:30.701263 139962008799040 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1003 09:21:30.768834 139962008799040 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W1003 09:21:32.049101 139962008799040 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Istanbul_weights_e04_valoss0.009_relu.hdf5
· data_split: all_possible_slots
· model_type: ConvLSTM
· city      : Istanbul
 1 epoch training is 3427 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Istanbul shape: (7, 3, 3)
mean mse: 0.009236385
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.1224265e-05' '0.0031339214' '0.024371097']
 ['10 minutes' '8.866059e-05' '0.0031522785' '0.024463298']
 ['15 minutes' '9.32195e-05' '0.003189214' '0.024554538']]
-----------------------------
mean mse like submission: 0.009235346
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '8.153998e-05' '0.0031343412' '

In [3]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

W1004 15:25:18.903795 140116142995264 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1004 15:25:18.905065 140116142995264 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1004 15:25:18.905945 140116142995264 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1004 15:25:18.972841 140116142995264 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W1004 15:25:20.292831 140116142995264 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Istanbul_weights_e04_valoss0.009_relu2.hdf5
· data_split: all_possible_slots
· model_type: ConvLSTM
· city      : Istanbul
 1 epoch training is 3427 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Istanbul shape: (7, 3, 3)
mean mse: 0.009111477
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '7.8989e-05' '0.0030745196' '0.024060233']
 ['10 minutes' '8.542469e-05' '0.0030886128' '0.024156224']
 ['15 minutes' '8.966214e-05' '0.0031202328' '0.024249407']]
-----------------------------
mean mse like submission: 0.009109962
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '7.9298465e-05' '0.0030749415' 

In [4]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

W1005 18:33:51.287383 140681566517056 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1005 18:33:51.288608 140681566517056 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1005 18:33:51.289451 140681566517056 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1005 18:33:51.362081 140681566517056 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W1005 18:33:52.643657 140681566517056 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Istanbul_weights_e03_valoss0.009_relu3.hdf5
· data_split: all_possible_slots
· model_type: ConvLSTM
· city      : Istanbul
 1 epoch training is 3427 batches, validation is 7, and test is 72.
Evaluating time-bin/time-slot batch 1/7
Evaluating time-bin/time-slot batch 2/7
Evaluating time-bin/time-slot batch 3/7
Evaluating time-bin/time-slot batch 4/7
Evaluating time-bin/time-slot batch 5/7
Evaluating time-bin/time-slot batch 6/7
Evaluating time-bin/time-slot batch 7/7
city: Istanbul shape: (7, 3, 3)
mean mse: 0.009066772
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '7.8933466e-05' '0.0030429643' '0.023945691']
 ['10 minutes' '8.49807e-05' '0.0030559893' '0.024062108']
 ['15 minutes' '8.90371e-05' '0.0030893798' '0.024151865']]
-----------------------------
mean mse like submission: 0.009066001
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '7.9236386e-05' '0.003043327' 

#### I2: ConvLSTM+Clf

In [5]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 1, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs new: /home/pherruzo/projects/nips_traffic/checkpoints/weights_Istanbul_06-1.04.hdf5
temporal
· data_split: non-overlapping
· model_type: ConvLSTM+Clf
· city      : Istanbul
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Istanbul shape: (14, 3, 3)
mean mse: 0.0096311355
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 mi

### Berlin
#### B1: ConvLSTM

In [16]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 2
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Berlin_model_ep_5.h5
· data_split: non-overlapping
· model_type: ConvLSTM
· city      : Berlin
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Berlin shape: (14, 3, 3)
mean mse: 0.007144515
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 minutes' '0.00024783806' '0.00545

#### B2: ConvLSTM+Clf

In [17]:
# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf", 2: "RAE_w_SC"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 1, 2
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
model_evaluate(val_ds, model, training_params["log_path"], training_params["city"], training_params["model_type"])

-->> Data has been shuffled in: training
loading weitghs new: /home/pherruzo/projects/nips_traffic/checkpoints/Berlin_model_ep_1_reg_clf_cont.h5
temporal
· data_split: non-overlapping
· model_type: ConvLSTM+Clf
· city      : Berlin
 1 epoch training is 3420 batches, validation is 14, and test is 15.
Evaluating time-bin/time-slot batch 1/14
Evaluating time-bin/time-slot batch 2/14
Evaluating time-bin/time-slot batch 3/14
Evaluating time-bin/time-slot batch 4/14
Evaluating time-bin/time-slot batch 5/14
Evaluating time-bin/time-slot batch 6/14
Evaluating time-bin/time-slot batch 7/14
Evaluating time-bin/time-slot batch 8/14
Evaluating time-bin/time-slot batch 9/14
Evaluating time-bin/time-slot batch 10/14
Evaluating time-bin/time-slot batch 11/14
Evaluating time-bin/time-slot batch 12/14
Evaluating time-bin/time-slot batch 13/14
Evaluating time-bin/time-slot batch 14/14
city: Berlin shape: (14, 3, 3)
mean mse: 0.0071446365
[['        ' '    Speed   ' '   Volume  ' '   Heading  ']
 [' 5 mi

In [36]:
training_params["output_dir"] = '/home/pherruzo/data/nips_traffic_submissions/raewscwwtwi'
training_params["output_dir"]

'/home/pherruzo/data/nips_traffic_submissions/raewscwwtwi'

## Generate submission files


In [37]:

write_submission_files(test_ds, model, training_params["output_dir"], training_params["city"], training_params["model_type"])

City:Moscow, just wrote file 1/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20181031_100m_bins.h5
City:Moscow, just wrote file 2/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20180323_100m_bins.h5
City:Moscow, just wrote file 3/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20181216_100m_bins.h5
City:Moscow, just wrote file 4/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20180415_100m_bins.h5
City:Moscow, just wrote file 5/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20180724_100m_bins.h5
City:Moscow, just wrote file 6/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20180731_100m_bins.h5
City:Moscow, just wrote file 7/72: /home/pherruzo/data/nips_traffic_submissions/raewscwwtwi/Moscow/Moscow_test/20180716_100m_bins.h5
City:Moscow, just wrote file 8/72: /home/pherruzo/data/nips_traffic_s

In [4]:
# !rm -rf /home/pherruzo/data/nips_traffic_submissions/conv_clf/Moscow/Moscow_test/

# load data and model --> {0: "ConvLSTM", 1: "ConvLSTM+Clf"}, {0: 'Moscow', 1: 'Istanbul', 2: 'Berlin'}
m_type, city = 0, 1
training_ds, val_ds, test_ds, model, training_params = load_model_and_data(MODEL_TYPES[m_type], CITIES[city], params)

# eval
write_submission_files(test_ds, model, training_params["output_dir"], training_params["city"], training_params["model_type"])

W1004 15:44:40.881412 139768439117632 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1004 15:44:40.882701 139768439117632 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1004 15:44:40.883593 139768439117632 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1004 15:44:40.952800 139768439117632 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

-->> Data has been shuffled in: training


W1004 15:44:42.229185 139768439117632 deprecation_wrapper.py:119] From /home/pherruzo/anaconda3/envs/nips/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loading weitghs: /home/pherruzo/projects/nips_traffic/checkpoints/Istanbul_weights_e04_valoss0.009_relu2.hdf5
· data_split: all_possible_slots
· model_type: ConvLSTM
· city      : Istanbul
 1 epoch training is 3427 batches, validation is 7, and test is 72.
City:Istanbul, just wrote file 1/72: /home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt/Istanbul/Istanbul_test/20180919_100m_bins.h5
City:Istanbul, just wrote file 2/72: /home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt/Istanbul/Istanbul_test/20180522_100m_bins.h5
City:Istanbul, just wrote file 3/72: /home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt/Istanbul/Istanbul_test/20181006_100m_bins.h5
City:Istanbul, just wrote file 4/72: /home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt/Istanbul/Istanbul_test/20180417_100m_bins.h5
City:Istanbul, just wrote file 5/72: /home/pherruzo/data/nips_traffic_submissions/conv_clf_mixt/Istanbul/Istanbul_test/20180518_100m_bins.h5
City:Istanbul, just wrote file 6/72: /